In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
import torch
from sklearn.utils.class_weight import compute_class_weight

In [2]:
import re

def super_clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    
    # 1. Транслит, как и было
    cyrillic_map = str.maketrans("abvgdeziklmnoprstufhcy", "абвгдезиклмнопрстуфхцы")
    text = text.translate(cyrillic_map)
    text = text.replace('blia', 'бля').replace('blya', 'бля').replace('ebat', 'ебать')

    # 2. Агрессивное удаление разделителей между буквами
    # Это найдет "б л я т ь", "б.л.я.т.ь", "б-л.я т.ь" и т.д.
    # Мы ищем букву, за которой следует до 3-х символов-разделителей, и потом снова буква.
    # И так повторяем.
    def merge_separated_letters(match):
        return match.group(0).replace('.', '').replace(' ', '').replace('-', '').replace('!', '').replace('@', '').replace('#', '').replace('$', '').replace('*', '')

    text = re.sub(r'([а-яё])([\s\.\-]+[а-яё]){2,}', merge_separated_letters, text)

    # 3. Нормализация "е" и "ё"
    text = text.replace('ё', 'е')
    
    # 4. Удаление повторяющихся букв (растянутость)
    text = re.sub(r'([а-я])\1+', r'\1', text)
    
    # 5. Более умные замены звездочек и пропусков.
    # Заменяем все, что похоже на мат, на его корень.
    text = re.sub(r'х[уеё*@#$ ]{1,5}[йяиюе]', 'хуй', text)
    text = re.sub(r'п[иеё*@#$ ]{1,5}[зс][д]', 'пизд', text)
    text = re.sub(r'[её][б*@#$ ]{1,5}[аоуя]', 'еб', text)
    text = re.sub(r'б[л*@#$ ]{1,5}[я]', 'бля', text)
    text = re.sub(r'м[у*@#$ ]{1,5}[д][аеио]', 'муд', text)
    
    return text


In [3]:
import random

def augment_profanity(df):
    profanity_samples = df[(df['label'] == 1) & (df['text'].str.contains(r'хуй|пизд|еб|бля|муд'))].copy()
    if profanity_samples.empty:
        return df

    new_rows = []
    
    for _, row in profanity_samples.iterrows():
        text = row['text']
        
        # Вариант 1: Растягивание букв
        if random.random() < 0.5:
            words = text.split()
            if not words: continue
            word_to_stretch = random.choice(words)
            if len(word_to_stretch) > 2:
                char_to_stretch = random.choice(word_to_stretch)
                stretched_word = word_to_stretch.replace(char_to_stretch, char_to_stretch * random.randint(3, 7))
                new_text = text.replace(word_to_stretch, stretched_word)
                new_rows.append({'text': new_text, 'label': 1})

        # Вариант 2: Вставка разделителей
        if random.random() < 0.5:
            words = text.split()
            if not words: continue
            word_to_separate = random.choice(words)
            if len(word_to_separate) > 2:
                separator = random.choice(['.', ' ', '-', '*'])
                separated_word = separator.join(list(word_to_separate))
                new_text = text.replace(word_to_separate, separated_word)
                new_rows.append({'text': new_text, 'label': 1})

    if not new_rows:
        return df
        
    augmented_df = pd.DataFrame(new_rows)
    return pd.concat([df, augmented_df], ignore_index=True)


In [4]:
# --- 1. Настройки и загрузка данных -1--
MODEL_NAME = "cointegrated/rubert-tiny2" # Общая модель для всех, быстрая и качественная
MAX_LENGTH = 256 # Длина текста, можно увеличить до 512, если отзывы длинные
BATCH_SIZE = 16 # Уменьшайте до 8, если не хватает видеопамяти
EPOCHS = 1 # 1 эпоха - для файн-тьюна

print("Загрузка данных...")
train_df = pd.read_csv('train.csv')
#test_df = pd.read_csv('test.csv')

# ШАГ 1: АУГМЕНТАЦИЯ
# Сначала аугментируем на "чистых" данных, чтобы создать "грязные" примеры
print("Аугментация данных...")
# Запускаем аугментацию несколько раз, чтобы создать больше примеров
augmented_train_df = train_df.copy()
for i in range(5): # 3 прохода аугментации
    print(f"Проход аугментации №{i+1}")
    augmented_train_df = augment_profanity(augmented_train_df)
print(f"Размер датасета после аугментации: {len(augmented_train_df)}")

print("Применение супер-очистки к данным...")
augmented_train_df['text'] = augmented_train_df['text'].apply(super_clean_text)

train_df = augmented_train_df

# Очистка от пустых строк на всякий случай
train_df.dropna(subset=['text'], inplace=True)
train_df['text'] = train_df['text'].astype(str)
#test_df['text'] = test_df['text'].astype(str)

print("Разделение на train/eval...")
# Стратифицированное разделение для сохранения баланса классов
train_subset_df, eval_df = train_test_split(
    train_df,
    test_size=0.15, # 15% на валидацию
    random_state=42,
    stratify=train_df['label']
)

# Оставим полный train_df для финального обучения
# train_subset_df - для быстрой проверки гипотез

train_dataset = Dataset.from_pandas(train_subset_df)
eval_dataset = Dataset.from_pandas(eval_df)
full_train_dataset = Dataset.from_pandas(train_df) # Для финального обучения
#test_dataset = Dataset.from_pan     das(test_df)

print("Инициализация токенизатора...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH
    )

print("Токенизация данных...")
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)
tokenized_full_train = full_train_dataset.map(tokenize_function, batched=True)
#tokenized_test = test_dataset.map(tokenize_function, batched=True)


Загрузка данных...
Аугментация данных...
Проход аугментации №1
Проход аугментации №2
Проход аугментации №3
Проход аугментации №4
Проход аугментации №5
Размер датасета после аугментации: 324122
Применение супер-очистки к данным...
Разделение на train/eval...
Инициализация токенизатора...
Токенизация данных...


Map:   0%|          | 0/275503 [00:00<?, ? examples/s]

Map:   0%|          | 0/48619 [00:00<?, ? examples/s]

Map:   0%|          | 0/324122 [00:00<?, ? examples/s]

In [5]:
# --- 2. Функция для метрики (общая для всех) ---
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='binary') # 'binary' для F1 по классу 1
    acc = accuracy_score(labels, predictions)
    return {"f1": f1, "accuracy": acc}

In [6]:
# --- 3. Общие аргументы для обучения ---
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True, # Очень важно!
    metric_for_best_model="f1", # Оптимизируемся по F1
    greater_is_better=True,
    fp16=torch.cuda.is_available(), # Автоматическое вкл. смешанной точности, если есть GPU
)

In [7]:
# --- Пайплайн 2: BERT с Weighted Loss ---
print("\n--- Запуск Пайплайна 2: BERT с Weighted Loss ---")
import torch.nn as nn

# --- Расчет весов для классов ---
print("Расчет весов для классов...")
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['label']),
    y=train_df['label']
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to("cuda" if torch.cuda.is_available() else "cpu")
print(f"Веса для классов: {class_weights_tensor}")


--- Запуск Пайплайна 2: BERT с Weighted Loss ---
Расчет весов для классов...
Веса для классов: tensor([0.7700, 1.4258], device='cuda:0')


In [8]:
class WeightedLossTrainer(Trainer):
    # Добавлен **kwargs для совместимости с новыми версиями Trainer
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Извлечение меток
        labels = inputs.pop("labels")

        # Прямой проход модели
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Использование весов классов (предполагается, что class_weights_tensor определен)
        # Убедитесь, что class_weights_tensor находится на том же устройстве (CPU/CUDA), что и logits
        # class_weights_tensor = class_weights_tensor.to(logits.device)
        loss_fct = nn.CrossEntropyLoss(weight=class_weights_tensor)

        # Вычисление взвешенных потерь
        loss = loss_fct(
            logits.view(-1, self.model.config.num_labels),
            labels.view(-1)
        )

        return (loss, outputs) if return_outputs else loss

In [9]:
# --- Кастомный Trainer ---
import os


# Сделайте так:
# --- ИЗМЕНЕНИЯ ЗДЕСЬ ---
# Указываем относительный путь как и раньше
relative_path = "./final_weighted_model-1"
# Преобразуем его в полный (абсолютный) путь, понятный системе
SAVED_MODEL_PATH = os.path.abspath(relative_path)

print(f"Загрузка уже обученной модели из {SAVED_MODEL_PATH} для продолжения обучения...")
# Теперь from_pretrained поймет, что это локальная папка, а не репозиторий на хабе
model_weighted = AutoModelForSequenceClassification.from_pretrained(SAVED_MODEL_PATH, num_labels=2, local_files_only=True)
trainer_weighted = WeightedLossTrainer(
    model=model_weighted,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
)

print("Обучение модели с весами...")
trainer_weighted.train()

print("Оценка модели с весами на валидации...")


Загрузка уже обученной модели из C:\Users\malan\PycharmProjects\PythonProject\final_weighted_model-1 для продолжения обучения...
Обучение модели с весами...


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.031000,0.049812,0.985034,0.989531


Оценка модели с весами на валидации...


In [10]:
eval_results_weighted = trainer_weighted.evaluate()
print(f"Результаты валидации Weighted Loss: {eval_results_weighted}")


Результаты валидации Weighted Loss: {'eval_loss': 0.04981185868382454, 'eval_f1': 0.9850342536238276, 'eval_accuracy': 0.9895308418519508, 'eval_runtime': 79.7776, 'eval_samples_per_second': 609.432, 'eval_steps_per_second': 38.093, 'epoch': 1.0}


In [11]:
# --- Обучение на всех данных и предсказание ---
print("Переобучение модели с весами на всех данных...")

# Создаем копию аргументов для финального обучения
final_training_args_dict = training_args.to_dict()

# --- ВНОСИМ ИЗМЕНЕНИЯ ДЛЯ ФИНАЛЬНОГО ОБУЧЕНИЯ ---
# Отключаем оценку, т.к. eval_dataset не будет
final_training_args_dict['eval_strategy'] = 'no'
# Также отключаем сохранение по эпохам (сохраним один раз в конце)
final_training_args_dict['save_strategy'] = 'no'
# И загрузку лучшей модели, т.к. нет метрики для выбора
final_training_args_dict['load_best_model_at_end'] = False

# Преобразуем словарь обратно в объект TrainingArguments
final_args = TrainingArguments(**final_training_args_dict)

# Создаем финальный тренер с НОВЫМИ аргументами
final_trainer_weighted = WeightedLossTrainer(
    model=model_weighted,   # Используем уже дообученную модель с первого этапа!
    args=final_args,        # Используем новые аргументы без валидации
    train_dataset=tokenized_full_train,
    # eval_dataset и compute_metrics здесь уже не нужны
)

# Запускаем финальное дообучение
print("Запуск финального дообучения...")
final_trainer_weighted.train()

print("Сохранение финальной модели...")
# Сохраняем модель и токенизатор в одну папку
SAVE_PATH = "./final_model_after_cleaning-2"
final_trainer_weighted.save_model(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
print(f"Финальная модель и токенизатор сохранены в '{SAVE_PATH}'")
# --- Обучение на всех данных и предсказание ---
# print("Переобучение модели с весами на всех данных...")
# final_trainer_weighted = WeightedLossTrainer(
#     model=model_weighted,
#     args=training_args,
#     train_dataset=tokenized_full_train,
# )
# final_trainer_weighted.train()

#print("Предсказание для test.csv...")
#predictions_w, _, _ = final_trainer_weighted.predict(tokenized_test)
#predicted_labels_w = np.argmax(predictions_w, axis=1)

#submission_weighted = pd.DataFrame({'id': test_df['id'], 'label': predicted_labels_w})
#submission_weighted.to_csv('submission_weighted.csv', index=False)
#print("Файл submission_weighted.csv готов!")


Переобучение модели с весами на всех данных...
Запуск финального дообучения...


C:\Users\malan\PycharmProjects\PythonProject\.venv\Lib\site-packages\transformers\training_args.py:2093: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


Step,Training Loss
100,0.034800
200,0.047800
300,0.049000
400,0.036300
500,0.035500
600,0.037600
700,0.050900
800,0.042000
900,0.044300
1000,0.052700


Сохранение финальной модели...
Финальная модель и токенизатор сохранены в './final_model_after_cleaning-2'


In [12]:
print("Сохранение финальной модели...")
# Сохраняем модель и токенизатор в одну папку
SAVE_PATH = "./final_model_after_cleaning-2-15-08"
final_trainer_weighted.save_model(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
print(f"Финальная модель и токенизатор сохранены в '{SAVE_PATH}'")

Сохранение финальной модели...
Финальная модель и токенизатор сохранены в './final_model_after_cleaning-2-15-08'


In [13]:
# # --- Обучение на всех данных и предсказание ---
# print("Переобучение модели с весами на всех данных...")
#
# # Создаем копию аргументов для финального обучения
# final_training_args = training_args.to_dict()
#
# # Отключаем оценку и сохранение по шагам, т.к. eval_dataset не будет
# final_training_args['eval_strategy'] = 'no'
# final_training_args['save_strategy'] = 'no'
# # Также отключаем загрузку лучшей модели, т.к. нет метрики для выбора
# final_training_args['load_best_model_at_end'] = False
#
# # Преобразуем словарь обратно в TrainingArguments
# final_args = TrainingArguments(**final_training_args)
#
#
# # Создаем финальный тренер с новыми аргументами
# final_trainer_weighted = WeightedLossTrainer(
#     model=model_weighted,   # Используем уже дообученную и лучшую модель с первого этапа!
#     args=final_args,        # Используем новые аргументы без валидации
#     train_dataset=tokenized_full_train,
#     # eval_dataset и compute_metrics здесь не нужны
# )
#
# # Запускаем финальное дообучение
# final_trainer_weighted.train()
#
# print("Сохранение финальной модели...")
# # Модель сохраняется в папку, указанную в output_dir
# final_trainer_weighted.save_model("./final_weighted_model")
# tokenizer.save_pretrained("./final_weighted_model") # Сохраняем и токенизатор рядом
#
# print("Предсказание для test.csv...")
# predictions_w = final_trainer_weighted.predict(tokenized_test)
# predicted_labels_w = np.argmax(predictions_w.predictions, axis=1)
#
# submission_weighted = pd.DataFrame({'id': test_df['id'], 'label': predicted_labels_w})
# submission_weighted.to_csv('submission_weighted.csv', index=False)
# print("Файл submission_weighted.csv и модель в ./final_weighted_model готовы!")